# Problem Statement

Welcome to your first week of work at the Disease And Treatment Agency, division of Societal Cures In Epidemiology and New Creative Engineering (DATA-SCIENCE). Time to get to work!

Due to the recent epidemic of West Nile Virus in the Windy City, we've had the Department of Public Health set up a surveillance and control system. We're hoping it will let us learn something from the mosquito population as we collect data over time. Pesticides are a necessary evil in the fight for public health and safety, not to mention expensive! We need to derive an effective plan to deploy pesticides throughout the city, and that is exactly where you come in!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#train_longs = train['Longitude']
#train_lats = train['Latitude']

#spray_longs = spray['Longitude']
#spray_lats = spray['Latitude']

In [3]:
#plt.figure(figsize=(15,20))
#origin = [41.6, -88.0]              # lat/long of origin (lower left corner)
#upperRight = [42.1, -87.5]          # lat/long of upper right corner

#mapdata = np.loadtxt("./assets/mapdata_copyright_openstreetmap_contributors.txt")

#intersection = [41.909614, -87.746134]  # co-ordinates of intersection of IL64 / IL50 according to Google Earth


# generate plot
#plt.imshow(mapdata, cmap=plt.get_cmap('gray'), extent=[origin[1], upperRight[1], origin[0], upperRight[0]])
#plt.scatter(x=spray_longs, y=spray_lats, c='r', s=20)
#plt.scatter(x=train_longs, y=train_lats, c='b', s=60, marker='s')

#plt.show()
#plt.savefig('map.png')

In [4]:
#plt.figure(figsize=(15,20))
#origin = [41.6, -88.0]              # lat/long of origin (lower left corner)
#upperRight = [42.1, -87.5]          # lat/long of upper right corner

#mapdata = np.loadtxt("./assets/mapdata_copyright_openstreetmap_contributors.txt")


# generate some data to overlay
#numPoints = 50
#lats = (upperRight[0] - origin[0]) * np.random.random_sample(numPoints) + origin[0]
#longs = (upperRight[1] - origin[1]) * np.random.random_sample(numPoints) + origin[1]

#intersection = [41.909614, -87.746134]  # co-ordinates of intersection of IL64 / IL50 according to Google Earth


# generate plot
#plt.imshow(mapdata, cmap=plt.get_cmap('gray'), extent=[origin[1], upperRight[1], origin[0], upperRight[0]])
#plt.scatter(x=longs, y=lats, c='r', s=20)
#plt.scatter(x=intersection[1], y=intersection[0], c='b', s=60, marker='s')

#plt.show()
#plt.savefig('map.png')

# Data Cleaning

In [5]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 30)

In [6]:
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
weather = pd.read_csv('../assets/weather.csv')
spray = pd.read_csv('../assets/spray.csv')

## Train

In [7]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [8]:
# check for null values
train.isnull().sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

## Weather

In [9]:
weather.info()
# check what columns to use, what to drop?
    # PrecipTotal, Water1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

In [20]:
# check for missing values
weather.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [21]:
# check for duplicates
weather.duplicated().sum()

0

In [39]:
# check for missing or trace values based on data dict
weather.isin(['T', 'M']).sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg             11
Depart         1472
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise        1472
Sunset         1472
CodeSum           0
Depth          1472
Water1         2944
SnowFall       1472
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [ ]:
# we would need to impute the missing data

## Spray

In [10]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


We see that in our train and test datasets, the observations do not have a time column. In this case, we will be dropping this column.

In [11]:
# drop time column in spray since train data doesn't have time column
spray.drop(columns='Time', inplace=True)

In [12]:
# check for duplicates
spray[spray.duplicated(subset=['Date', 'Latitude', 'Longitude'])]

,Date,Latitude,Longitude
485,2011-09-07,41.983917,-87.793088
490,2011-09-07,41.986460,-87.794225
491,2011-09-07,41.986460,-87.794225
492,2011-09-07,41.986460,-87.794225
493,2011-09-07,41.986460,-87.794225
494,2011-09-07,41.986460,-87.794225
495,2011-09-07,41.986460,-87.794225
496,2011-09-07,41.986460,-87.794225
497,2011-09-07,41.986460,-87.794225
498,2011-09-07,41.986460,-87.794225


In [13]:
spray.duplicated(subset=['Date', 'Latitude', 'Longitude']).sum()

541

In [14]:
spray.shape

(14835, 3)

In [15]:
spray.drop_duplicates(inplace=True)

In [17]:
spray.shape

(14294, 3)

In [18]:
# convert date column from object to datetime
spray['Date'] = pd.to_datetime(spray['Date'], format='%Y-%m-%d')

# Exploratory Data Analysis

# Modelling

# Conclusion